**This file is only used to compare the result of mymd with that of torchmd.**

In [1]:
import os
import numpy as np
import torch
import random
import parmed
import nglview as ng
import MDAnalysis as md
from MDAnalysis.analysis import dihedrals
import matplotlib.pyplot as plt

import mymd

In [2]:
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
data_path = 'data/prod_alanine_dipeptide_amber_torchmd/'

prmtop_file = os.path.join(data_path, 'structure.prmtop')
pdb_file = os.path.join(data_path, 'structure.pdb')
coor_file = os.path.join(data_path, 'input.coor')
xsc_file = os.path.join(data_path, 'input.xsc')

cutoff = 9
T = 300
min_energy_max_iter = 100
device = 'cpu'
precision = torch.float

In [4]:
u = md.Universe(pdb_file)
ng.show_mdanalysis(u, gui=True)

NGLWidget()

In [5]:
mol = mymd.get_molecule(prmtop_file=prmtop_file, coor_file=coor_file, xsc_file=xsc_file)

print('---coords---')
print(mol.coords[:2])
print('-----box-----')
print(mol.box)
print('---charge---')
print(mol.charge[:3])
print('---masses---')
print(mol.masses[:3])
print('----bonds----')
print(mol.bonds[:4])
print('---bonds.shape---')
print(mol.bonds.shape)

---coords---
[[[ 8.487608 ]
  [ 8.890593 ]
  [13.186471 ]]

 [[ 8.638412 ]
  [ 9.8404045]
  [12.673427 ]]]
-----box-----
[[19.83881]
 [19.6193 ]
 [19.6342 ]]
---charge---
[ 0.1123 -0.3662  0.1123]
---masses---
[ 1.008 12.01   1.008]
----bonds----
[[ 1  2]
 [ 1  3]
 [ 0  1]
 [10 11]]
---bonds.shape---
(687, 2)


In [6]:
try:
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=False)
except:
    print('False causes error, use True.')
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=True)

print(ff.mol.atomtype[:10])
print(ff.mol.charge[:10])
print(set(ff.mol.atomtype))
print(set(ff.params.atom_types))

['HC' 'CT' 'HC' 'HC' 'C' 'O' 'N' 'H' 'CX' 'H1']
[ 0.1123 -0.3662  0.1123  0.1123  0.5972 -0.5679 -0.4157  0.2719  0.0337
  0.0823]
{'HC', 'H', 'O', 'CT', 'OW', 'CX', 'HW', 'C', 'H1', 'N'}
{'HC', 'H', 'O', 'CT', 'OW', 'CX', 'HW', 'C', 'H1', 'N'}


In [7]:
system = mymd.System(mol, ff, cutoff=cutoff)

print(system.bond_params[:5])
print(system.improper_params[0]['params'][:5])
print(system.uni_atom_types == np.unique(mol.atomtype))
print(system.uni_atom_types[system.mapped_atom_types[:6]] == mol.atomtype[:6])
print(system.box, mol.box)
print(system.cutoff)
print(system.external)
print(np.array(system.get_exclusions(types=['bonds']))[:3])
print(np.all(np.vstack((np.array(system.bonds), np.array(system.angles[:, [0, 2]]))) == np.array(system.get_exclusions(types=['bonds', 'angles']))))

tensor([[340.0000,   1.0900],
        [340.0000,   1.0900],
        [340.0000,   1.0900],
        [317.0000,   1.5220],
        [570.0000,   1.2290]])
tensor([[10.5000,  3.1416,  2.0000],
        [ 1.1000,  3.1416,  2.0000],
        [10.5000,  3.1416,  2.0000],
        [ 1.1000,  3.1416,  2.0000]])
[ True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True]
tensor([19.8388, 19.6193, 19.6342]) [[19.83881]
 [19.6193 ]
 [19.6342 ]]
9
None
[[0 1]
 [1 2]
 [1 3]]
True


In [8]:
integrator = mymd.VelocityVerletIntegrator(1)
print(integrator.dt, integrator.dt_fs)

0.02045482949774598 1


In [9]:
simulation = mymd.Simulation(mol, system, integrator, device, precision)
simulation.set_positions(mol.coords)
simulation.set_velocities_to_temperature(T=T)
simulation.update_potentials_and_forces()

print(torch.abs(simulation.vel).mean())
print(simulation.forces.shape)

# compared with torchmd (in its github repo, examples/tutorial.ipynb)
# need some modifications in its code:
# forces = Forces(parameters, cutoff=9, rfa=False, switch_dist=None, terms=["bonds", "angles", "dihedrals", "impropers", "electrostatics", "lj"], exclusions=('bonds', 'angles'))
print(simulation.forces[:5])
print(simulation.forces[-3:])
print(simulation.potentials)
print(simulation.potentials_sum)

tensor(0.4706)
torch.Size([688, 3])
tensor([[  2.1204,   1.6945,   3.8852],
        [-14.4477, -15.9356,   2.9400],
        [  2.2896,   2.8798,  -3.1538],
        [  1.4504,   9.1638,  -0.7600],
        [-16.9996,  23.9501, -24.9862]])
tensor([[-28.8178,   7.0773,  34.4423],
        [  1.8655,  -8.3980, -16.7160],
        [ 20.7306,  -3.3352,  -9.9856]])
{'bonds': 3.957749366760254, 'angles': 2.8445701599121094, 'dihedrals': 10.579872131347656, 'impropers': 1.2417112588882446, 'lj': 358.54205322265625, 'electrostatics': -2686.371337890625}
-2309.2053817510605


In [10]:
# compared with torchmd (in its github repo, examples/tutorial.ipynb)
# need to use the modification of code mentioned in the previous block
print(simulation.pos[:5])
simulation.minimize_energy(min_energy_max_iter)
print(simulation.pos[:5])

print(simulation.forces[:5])
print(simulation.potentials)
print(simulation.potentials_sum)

tensor([[ 8.4876,  8.8906, 13.1865],
        [ 8.6384,  9.8404, 12.6734],
        [ 8.3185, 10.6384, 13.3434],
        [ 8.0228,  9.7510, 11.7783],
        [10.0775,  9.9498, 12.2872]])
num_fun_eval	E_pot    	norm_F_max	rms      	max_abs_f_1d
     0      	-2309.205382	64.178003	14.611752	62.617146
     1      	-1631.939162	160.349691	37.092344	130.381134
     2      	-2561.768508	42.287161	9.231950	41.672520
     3      	-2339.988267	25.866915	5.810489	23.489002
     4      	-2529.503574	39.209721	8.682197	38.655632
     5      	-2543.182295	42.131685	9.202882	41.520260
     6      	-2561.771691	42.286224	9.231783	41.671604
     7      	-2530.127932	42.183849	9.211232	41.571285
     8      	-2561.773317	42.285812	9.231745	41.671238
     9      	-2561.773656	42.285812	9.231732	41.671238
     10     	-2561.989436	42.235690	9.222435	41.622070
     11     	-2530.055249	42.201907	9.214392	41.588936
     12     	-2561.989464	42.235690	9.222433	41.622070
     13     	-2536.421184	42.212627	9.